# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

## Лабораторная работа 5

### csv

In [16]:
import csv
import json
import pandas as pd
import numpy as np
from collections import defaultdict

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [63]:
with open('data/tags_sample.csv') as file:
    tags_file = csv.reader(file, delimiter=',')
    next(tags_file)
    tags_d = defaultdict(list)
    for row in tags_file:
        tags_d[int(row[0])].append(row[1])
with open('output_data/tags.sample.json', 'w') as file:
    json.dump(tags_d, file, indent=2)

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [71]:
recipes = pd.read_csv('data/recipes_sample_with_filled_nsteps.csv')
recipes.drop(recipes.columns[0], axis=1, inplace=True)
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22.0,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10.0,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7.0,this is a delicious soup that i originally fou...,NaN


In [72]:
n_tags = dict()
tags = dict()

for key, value in tags_d.items():
    n_tags[key] = len(value)
    tags[key] = ';'.join(value)
recipes['n_tags'] = recipes['id'].map(n_tags)
recipes['tags'] = recipes['id'].map(tags)
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0,25,weeknight;time-to-make;course;main-ingredient;...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN,31,15-minutes-or-less;time-to-make;course;prepara...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0,17,30-minutes-or-less;time-to-make;course;main-in...
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN,11,60-minutes-or-less;time-to-make;course;prepara...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,19,30-minutes-or-less;time-to-make;course;main-in...


1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

In [ ]:
with open('data/ingredients_sample.csv') as file:
    ingr_file = csv.DictReader(file, delimiter=',')
    header = next(ingr_file)
    
    id_ingr = defaultdict(list)
    
    for row in ingr_file:
        id_ingr[int(row['recipe_id'])].append(row['ingredient'])

id_ingr

1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

In [74]:
id_ingr_l = dict()
number_ingr = dict()

for key, value in id_ingr.items():
    id_ingr_l[key] = '*'.join(value)
    number_ingr[key] = len(value)

recipes['ingredients'] = recipes['id'].map(id_ingr_l)
recipes['n_ingredients'] = recipes['n_ingredients'].fillna(recipes['id'].map(number_ingr))
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags,ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0,25,weeknight;time-to-make;course;main-ingredient;...,carrot*onion*celery*broccoli stem*dried thyme*...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,3.0,31,15-minutes-or-less;time-to-make;course;prepara...,milk*frozen juice concentrate*plain yogurt
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0,17,30-minutes-or-less;time-to-make;course;main-in...,onion*frozen chopped spinach*eggs*garlic powde...
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,9.0,11,60-minutes-or-less;time-to-make;course;prepara...,sandwich bun*good seasonings italian salad dre...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,12.0,19,30-minutes-or-less;time-to-make;course;main-in...,beef steaks*vegetable oil*spicy mustard*fresh ...


1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

In [75]:
recipes['n_ingredients'].isnull().values.any()

False

In [76]:
recipes['n_ingredients'] = recipes['n_ingredients'].astype(int)

In [77]:
recipes.to_csv('output_data/recipes_sample_with_tags_ingredients.csv.csv')

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

In [78]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

In [79]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags,ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18,25,weeknight;time-to-make;course;main-ingredient;...,carrot*onion*celery*broccoli stem*dried thyme*...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,3,31,15-minutes-or-less;time-to-make;course;prepara...,milk*frozen juice concentrate*plain yogurt
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8,17,30-minutes-or-less;time-to-make;course;main-in...,onion*frozen chopped spinach*eggs*garlic powde...
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,9,11,60-minutes-or-less;time-to-make;course;prepara...,sandwich bun*good seasonings italian salad dre...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,12,19,30-minutes-or-less;time-to-make;course;main-in...,beef steaks*vegetable oil*spicy mustard*fresh ...


In [113]:
recipes['n_steps'] = recipes['n_steps'].astype('int')

In [ ]:
numeric_columns = ['id', 'minutes', 'contributor_id', 'n_steps', 'n_ingredients', 'n_tags']
recipes_l = recipes[recipes['submitted'].dt.year < 2000][numeric_columns].to_numpy()
recipes_g = recipes[recipes['submitted'].dt.year >= 2000][numeric_columns].to_numpy()
recipes_g

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

In [116]:
np.savez('output_data/two_recipes.npz', recipes_l, recipes_g)

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

In [ ]:
npzfile = np.load('output_data/two_recipes.npz')
npzfile['arr_0']

In [ ]:
recipes_l

In [126]:
npzfile.close()

### hdf

In [127]:
import h5py

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

In [ ]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

with h5py.File('data/nutrition_sample.h5', 'r') as f:
    f.visititems(print_attrs)

In [151]:
with h5py.File('data/nutrition_sample.h5', 'r') as f:
    print(f['dataset_0'].attrs['col_1'])

calories (#)


In [135]:
for key, val in obj.attrs.items():
        print(f"'{key}': '{val}'", end=' ')

<KeysViewHDF5 ['col_0', 'col_1']>


In [159]:
def print_info(name, obj):
    print(f'Dataset name={name}, dataset size={obj.shape}, metadata=\{{obj.attrs["col_1"]}\}')
    

with h5py.File('data/nutrition_sample.h5', 'r') as f:
    f.visititems(print_info)

SyntaxError: f-string: single '}' is not allowed (<ipython-input-159-89bf38377bcb>, line 2)

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Если датасет не содержитСохраните результаты в файл `nutrition_grouped.h5`

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.